# Read MRCONSO.RRF file
## Target subset of UMLS

In [1]:
import csv

# 定义文件路径  /DATA1/llm-research/2022AA-full/
mrconso_path = '../resource/Radiology/2024AA/META/MRCONSO.RRF'

concepts = {}

with open(mrconso_path, 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter='|')
    for row in reader:
        cui = row[0]
        concept_name = row[14]
        concepts[cui] = concept_name

print(len(concepts))


898720
